# Universal Perturbation on The PASCAL Visual Object Classes Challenge 2012 (VGG)

In [5]:
import sys
!{sys.executable} -m pip install torchvision torch sklearn pandas gdown 
!conda install -y -c conda-forge ipywidgets tqdm widgetsnbextension
!jupyter nbextension enable --py widgetsnbextension

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Using cached filelock-3.6.0-py3-none-any.whl (10.0 kB)
  Created wheel for gdown: filename=gdown-4.4.0-py3-none-any.whl size=14775 sha256=002b68d35a5f41efd60180f877dc6443fffe434e639e2d555da4a97a2ca3f7b5
  Stored in directory: /home/waris/.cache/pip/wheels/7b/7b/5d/656f46cd6889e4c93977be9586901d0adc1271b2d876c84c96
Successfully built gdown
You should consider upgrading via the '/home/waris/Projects/universal/universal/bin/python -m pip install --upgrade pip' command.
Solving environment: done

# All requested packages already installed.

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import _LRScheduler
import torch.utils.data as data
from torch.utils.data import Dataset

import torchvision.transforms as transforms
import torchvision.datasets as datasets

from sklearn import decomposition
from sklearn import manifold
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from tqdm.notebook import tqdm, trange
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

import copy
import random
import time
import os

## Defining the Model

In [ ]:
class VGG(nn.Module):
    def __init__(self, features, output_dim, device):
        super().__init__()
        self.device = device

        self.features = features

        self.avgpool = nn.AdaptiveAvgPool2d(7)

        self.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(4096, output_dim),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = x.to(self.device)
        h = x.view(x.shape[0], -1)
        x = self.classifier(h)
        return x, h
    
vgg11_config = [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M']

vgg13_config = [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512,
                512, 'M']

vgg16_config = [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512,
                'M', 512, 512, 512, 'M']

vgg19_config = [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512,
                512, 512, 'M', 512, 512, 512, 512, 'M']
    
def get_vgg_layers(config, batch_norm):

    layers = []
    in_channels = 3

    for c in config:
        assert c == 'M' or isinstance(c, int)
        if c == 'M':
            layers += [nn.MaxPool2d(kernel_size=2)]
        else:
            conv2d = nn.Conv2d(in_channels, c, kernel_size=3, padding=1)
            if batch_norm:
                layers += [conv2d, nn.BatchNorm2d(c), nn.ReLU(inplace=True)]
            else:
                layers += [conv2d, nn.ReLU(inplace=True)]
            in_channels = c

    return nn.Sequential(*layers)

vgg11_layers = get_vgg_layers(vgg11_config, batch_norm=True)
vgg11_layers

In [ ]:
OUTPUT_DIM = 20

model = VGG(vgg11_layers, OUTPUT_DIM, 'cuda')
# for freeze weight

model

In [ ]:
model.device

## Load pre-trained models

In [ ]:
import torchvision.models as models

pretrained_model = models.vgg11_bn(pretrained=True)
pretrained_model

In [ ]:
pretrained_model.classifier[-1]

In [ ]:
IN_FEATURES = pretrained_model.classifier[-1].in_features

final_fc = nn.Linear(IN_FEATURES, OUTPUT_DIM)
pretrained_model.classifier[-1] = final_fc
pretrained_model.classifier

In [ ]:
model.load_state_dict(pretrained_model.state_dict())

## Data Processing

In [13]:
ROOT = 'VOC2012'
ROOT_test = ROOT + "_test"

if not os.path.isdir(ROOT):
    !wget http://host.robots.ox.ac.uk/pascal/VOC/voc2012/VOCtrainval_11-May-2012.tar -O /tmp/VOC.tar
    !tar -xvf /tmp/VOC.tar
    !mv VOCdevkit/VOC2012 VOC2012

In [14]:
if not os.path.isdir(ROOT_test):
    !gdown --id 1gVllQTZdxA82byaZ_SjeOyjns0qnugwR -O /tmp/VOC_test.tar
    !tar -xvf /tmp/VOC_test.tar
    !mv VOCdevkit/VOC2012 VOC2012_test

In [35]:
pretrained_size = 224
pretrained_means = [0.485, 0.456, 0.406]
pretrained_stds = [0.229, 0.224, 0.225]

train_transforms = transforms.Compose([
                           transforms.Resize(pretrained_size),
                           transforms.RandomRotation(5),
                           transforms.RandomHorizontalFlip(0.5),
                           transforms.RandomCrop(pretrained_size, padding=10),
                           transforms.ToTensor(),
                           transforms.Normalize(mean=pretrained_means,
                                                std=pretrained_stds)
                       ])

test_transforms = transforms.Compose([
                           transforms.Resize(pretrained_size),
                           transforms.ToTensor(),
                           transforms.Normalize(mean=pretrained_means,
                                                std=pretrained_stds)
                       ])

In [43]:
import pandas as pd
from functools import reduce

def load_dataset(c):
    df = pd.read_csv(f"{ROOT}/ImageSets/Main/{c}_trainval.txt", delimiter=r"\s+", header=None, names=('id', c))
    return df

classes = ['bicycle', 'horse', 'cat', 'dog', 'pottedplant', 'aeroplane', 'sofa', 'bus', 'car', 'chair', 'bird', 'boat', 'cow', 'bottle', 'diningtable', 'person', 'tvmonitor', 'sheep', 'motorbike', 'train']
df = reduce(lambda df1,df2: pd.merge(df1,df2,on='id'), [load_dataset(c) for c in classes])
df['label'] = df.apply(lambda row: row[classes].tolist().index(1), axis=1)

with open(f"{ROOT}/ImageSets/Main/train.txt", 'r') as f:
    _ = f.read().split('\n')
    df['train'] = df.apply(lambda row: row.values[0] in _, axis=1)
train_data = df[df['train']][['id','label']]
valid_data = df[df['train'] == False][['id','label']]

print(len(train_data))
print(len(valid_data))
train_data

5717
5823


,id,label
3,2008_000008,1
5,2008_000015,13
7,2008_000019,3
9,2008_000023,13
12,2008_000028,8
...,...,...
11527,2011_003253,1
11529,2011_003255,15
11531,2011_003259,9
11537,2011_003274,19


In [44]:
def load_dataset_test(c):
    df = pd.read_csv(f"{ROOT_test}/ImageSets/Main/{c}_test.txt", delimiter=r"\s+", header=None, names=('id', c))
    return df

df_test = reduce(lambda df1,df2: pd.merge(df1,df2,on='id'), [load_dataset_test(c) for c in classes])
df_test['label'] = np.zeros(len(df_test), dtype=int)

test_data = df_test[['id', 'label']]
test_data

,id,label
0,2008_000001,0
1,2008_000004,0
2,2008_000005,0
3,2008_000006,0
4,2008_000010,0
...,...,...
10986,2011_003264,0
10987,2011_003265,0
10988,2011_003266,0
10989,2011_003267,0


In [45]:
class VOCDataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform
        self.labels = data['label'].tolist()
        self.image_paths = data['id'].tolist()
        
    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_filepath = f"{ROOT}/JPEGImages/{self.data.iloc[idx].values[0]}.jpg"
        image = Image.open(image_filepath)
        
        label = self.data.iloc[idx].values[1]
        if self.transform is not None:
            image = self.transform(image)
        return image, label

train_data = VOCDataset(train_data, train_transforms)
valid_data = VOCDataset(valid_data, test_transforms)
valid_data.dataset.transform = test_transforms
test_data  = VOCDataset(test_data, test_transforms)

AttributeError: 'VOCDataset' object has no attribute 'dataset'

In [41]:
def normalize_image(image):
    image_min = image.min()
    image_max = image.max()
    image.clamp_(min=image_min, max=image_max)
    image.add_(-image_min).div_(image_max - image_min + 1e-5)
    return image


def plot_images(images, labels, classes, normalize=True):

    n_images = len(images)

    rows = int(np.sqrt(n_images))
    cols = int(np.sqrt(n_images))

    fig = plt.figure(figsize=(10, 10))

    for i in range(rows*cols):

        ax = fig.add_subplot(rows, cols, i+1)

        image = images[i]

        if normalize:
            image = normalize_image(image)

        ax.imshow(image.permute(1, 2, 0).cpu().numpy())
        ax.set_title(classes[labels[i]])
        ax.axis('off')

In [ ]:
N_IMAGES = 25

images, labels = zip(*[(image, label) for image, label in
                       [train_data[i] for i in range(N_IMAGES)]])

plot_images(images, labels, classes)

In [ ]:
BATCH_SIZE = 64

train_iterator = data.DataLoader(train_data,
                                 shuffle=True,
                                 batch_size=BATCH_SIZE)

valid_iterator = data.DataLoader(valid_data,
                                 batch_size=BATCH_SIZE)

test_iterator = data.DataLoader(test_data,
                                batch_size=BATCH_SIZE)

## Training the model

In [ ]:
START_LR = 1e-7

optimizer = optim.Adam(model.parameters(), lr=START_LR)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# model.features

criterion = nn.CrossEntropyLoss()

model.classifier = model.classifier.to(device)
model = model.to(device)
criterion = criterion.to(device)

In [ ]:
from tqdm import tqdm

class LRFinder:
    def __init__(self, model, optimizer, criterion, device):

        self.optimizer = optimizer
        self.model = model
        self.criterion = criterion
        self.device = device

        torch.save(model.state_dict(), 'init_params.pt')

    def range_test(self, iterator, end_lr=10, num_iter=100,
                   smooth_f=0.05, diverge_th=5):

        lrs = []
        losses = []
        best_loss = float('inf')

        lr_scheduler = ExponentialLR(self.optimizer, end_lr, num_iter)

        iterator = IteratorWrapper(iterator)

        for iteration in tqdm(range(num_iter)):

            loss = self._train_batch(iterator)

            lrs.append(lr_scheduler.get_last_lr()[0])

            # update lr
            lr_scheduler.step()

            if iteration > 0:
                loss = smooth_f * loss + (1 - smooth_f) * losses[-1]

            if loss < best_loss:
                best_loss = loss

            losses.append(loss)

            if loss > diverge_th * best_loss:
                print("Stopping early, the loss has diverged")
                break

        # reset model to initial parameters
        model.load_state_dict(torch.load('init_params.pt'))

        return lrs, losses

    def _train_batch(self, iterator):

        self.model.train()

        self.optimizer.zero_grad()

        x, y = iterator.get_batch()

        x = x.to(self.device)
        y = y.to(self.device)

        y_pred, _ = self.model(x)

        loss = self.criterion(y_pred, y)

        loss.backward()

        self.optimizer.step()

        return loss.item()


class ExponentialLR(_LRScheduler):
    def __init__(self, optimizer, end_lr, num_iter, last_epoch=-1):
        self.end_lr = end_lr
        self.num_iter = num_iter
        super(ExponentialLR, self).__init__(optimizer, last_epoch)

    def get_lr(self):
        curr_iter = self.last_epoch
        r = curr_iter / self.num_iter
        return [base_lr * (self.end_lr / base_lr) ** r for base_lr in
                self.base_lrs]


class IteratorWrapper:
    def __init__(self, iterator):
        self.iterator = iterator
        self._iterator = iter(iterator)

    def __next__(self):
        try:
            inputs, labels = next(self._iterator)
        except StopIteration:
            self._iterator = iter(self.iterator)
            inputs, labels, *_ = next(self._iterator)

        return inputs, labels

    def get_batch(self):
        return next(self)

In [ ]:
END_LR = 10
NUM_ITER = 100

lr_finder = LRFinder(model, optimizer, criterion, device)
lrs, losses = lr_finder.range_test(train_iterator, END_LR, NUM_ITER)

In [ ]:
def plot_lr_finder(lrs, losses, skip_start=5, skip_end=5):

    if skip_end == 0:
        lrs = lrs[skip_start:]
        losses = losses[skip_start:]
    else:
        lrs = lrs[skip_start:-skip_end]
        losses = losses[skip_start:-skip_end]

    fig = plt.figure(figsize=(16, 8))
    ax = fig.add_subplot(1, 1, 1)
    ax.plot(lrs, losses)
    ax.set_xscale('log')
    ax.set_xlabel('Learning rate')
    ax.set_ylabel('Loss')
    ax.grid(True, 'both', 'x')
    plt.show()

In [ ]:
plot_lr_finder(lrs, losses, skip_start=10, skip_end=20)

In [ ]:
FOUND_LR = 5e-4

params = [
          {'params': model.features.parameters(), 'lr': FOUND_LR / 10},
          {'params': model.classifier.parameters()}
         ]

optimizer = optim.Adam(params, lr=FOUND_LR)

In [ ]:
def calculate_accuracy(y_pred, y):
    top_pred = y_pred.argmax(1, keepdim=True)
    correct = top_pred.eq(y.view_as(top_pred)).sum()
    acc = correct.float() / y.shape[0]
    return acc

In [ ]:
def train(model, iterator, optimizer, criterion, device):

    epoch_loss = 0
    epoch_acc = 0

    model.train()

    for (x, y) in tqdm(iterator, desc="Training", leave=False):

        x = x.to(device)
        y = y.to(device)

        optimizer.zero_grad()

        y_pred, _ = model(x)

        loss = criterion(y_pred, y)

        acc = calculate_accuracy(y_pred, y)

        loss.backward()

        optimizer.step()

        epoch_loss += loss.item()
        epoch_acc += acc.item()

    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion, device):

    epoch_loss = 0
    epoch_acc = 0

    model.eval()

    with torch.no_grad():

        for (x, y) in tqdm(iterator, desc="Evaluating", leave=False):

            x = x.to(device)
            y = y.to(device)

            y_pred, _ = model(x)

            loss = criterion(y_pred, y)

            acc = calculate_accuracy(y_pred, y)

            epoch_loss += loss.item()
            epoch_acc += acc.item()

    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
EPOCHS = 5

best_valid_loss = float('inf')


for epoch in trange(EPOCHS, desc="Epochs"):

    start_time = time.monotonic()

    train_loss, train_acc = train(model, train_iterator, optimizer, criterion, device)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion, device)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut4-model.pt')

    end_time = time.monotonic()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')